# Pré-processamento

In [ ]:
class MyObject:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, i):
        row = self.dataframe.iloc[i]
        return row['Value']

    def soma(self, i, value):
        self.dataframe.iloc[i]['Value'] = float(self.dataframe.iloc[i]['Value']+value)

In [ ]:
import pandas as pd

In [ ]:
distancias_centroides = pd.read_csv("matriz_dist_centroide_escola.csv", index_col=0)

In [ ]:
distancias_centroides = distancias_centroides.unstack().sort_index().unstack(level=0)

In [ ]:
distancias_centroides = distancias_centroides.sort_index()

In [ ]:
#distancias_centroides = distancias_centroides.melt(id_vars=['ID'], value_vars=distancias_centroides.columns[1:])
#distancias_centroides.rename(columns={'variable':'escola', 'value':'dist'}, inplace=True)

In [ ]:
#id_map = distancias_centroides['ID']
#escola_map = distancias_centroides['escola']

In [ ]:
#distancias_centroides['ID'] = distancias_centroides.index
#distancias_centroides['escola'] = distancias_centroides.index

In [ ]:
dicionario = {}
i = 0
for col in distancias_centroides.columns[:]:
  dicionario[col] = i
  i = i+1

In [ ]:
distancias_centroides.rename(columns=dicionario,inplace=True)

In [ ]:
id_map = distancias_centroides.index

In [ ]:
distancias_centroides = distancias_centroides.reset_index(drop=True)
distancias_centroides

In [ ]:
matriz_distancias = {}
for col in distancias_centroides.columns:
    matriz_distancias[col] = dict(zip(distancias_centroides.index, distancias_centroides[col]))

In [ ]:
matriz_distancias[0][0]

In [ ]:
capacidade_atual = pd.read_csv("capacidade-atual-escolas-REAL.csv")

In [ ]:
capacidade_atual = capacidade_atual.sort_values('CO_ENTIDAD')
capacidade_atual = capacidade_atual.reset_index(drop=True)

In [ ]:
id_capacidades = capacidade_atual['CO_ENTIDAD']

In [ ]:
matriz_capacidades = MyObject(capacidade_atual.rename(columns={'QT_MAT_FUN':'Value'}))

In [ ]:
demanda = pd.read_csv("populacao_setores_sjc_demanda.csv")

In [ ]:
demanda = demanda.sort_values('CD_GEOCODI')
demanda = demanda.reset_index(drop=True)

In [ ]:
id_demanda = demanda['CD_GEOCODI']

In [ ]:
matriz_demanda = MyObject(demanda.rename(columns={'POPULACAO':'Value'}))

In [ ]:
escolas_regiao = pd.read_csv("escolas-regiao-REAL.csv", delimiter=";", index_col='CO_ENTIDAD')
escolas_regiao = escolas_regiao.fillna(0)

In [ ]:
escolas_regiao

In [ ]:
escolas_regiao = escolas_regiao.sort_values('CO_ENTIDAD')
#escolas_regiao = escolas_regiao.reset_index(drop=True)

In [ ]:
escolas_regiao.loc[35121605]

In [ ]:
id_escolas_regiao = escolas_regiao.index
qtd_regioes = len(escolas_regiao.columns)

In [ ]:
escolas_regiao = escolas_regiao.reset_index(drop=True)

In [ ]:
dicionario_regioes = {}
i = 0
for col in escolas_regiao.columns[:]:
  dicionario_regioes[col] = i
  i = i+1

In [ ]:
escolas_regiao.rename(columns=dicionario_regioes,inplace=True)

In [ ]:
matriz_regioes = {}
for col in escolas_regiao.columns:
    matriz_regioes[col] = dict(zip(escolas_regiao.index, escolas_regiao[col]))

In [ ]:
matriz_regioes[0][0]+1

In [ ]:
# Extrai valores únicos para identificação
id_escolas = capacidade_atual.index.values
id_vertices = demanda.index.values

In [ ]:
#distancias_centroides_ampl = distancias_centroides.transpose().unstack()
#idx = distancias_centroides_ampl.index
#distancias_centroides_ampl.index = distancias_centroides_ampl.index.set_levels(idx.levels[1].astype("int64"), level=1)

In [ ]:
print(teitz_bart(matriz_distancias, 55))

# Teitz e Bart - do zero

In [ ]:
### PARÂMETROS ###
p = 55

In [ ]:
import random
import numpy as np
import copy
from timeit import default_timer as timer

INF = float('inf')

In [ ]:
# Testa restrição
def verifica_restricoes(median_x, median_y):
    start = timer()
    # R1: sum{j in escolas} x[j] = num_medianas;
    sum_escola = 0
    for j in range(len(median_x)):
      sum_escola += median_x[j]
    if sum_escola != p:
      print("ERRO NUM_MEDIANAS")
      return False

    # R3 {i in vertices}: sum{j in escolas} y[i,j] = 1;
    for i in range(len(id_vertices)):
      sum_escola = 0
      for j in range(len(median_x)):
        sum_escola += median_y[j][i]
      if (sum_escola != 1):
        print("ERRO MAIS DE UMA ESCOLA PRO MESMO VÉRTICE")
        return False


    '''
    # R4 {j in escolas}: capacidade[j]*x[j] - 0.25*sum{i in vertices}demanda[i]*y[i,j] >= 0;
    for j in range(len(median_x)):
      sum_vertice = 0
      for i in range(len(id_vertices)):
        sum_vertice += matriz_demanda[i]*median_y[j][i]
      restricao_1 = matriz_capacidades[j]*median_x[j] - 0.25*sum_vertice
      if (restricao_1 < 0):
        print("ERRO NA SOMA DAS CAPACIDADES")
        return False'''

    # R6 {k in regioes_sjc}: sum{j in escolas}x[j]*escola_regiao_sjc[j,k] >= 1;
    for k in range(qtd_regioes):
      sum_vertice = 0
      for j in range(len(id_escolas)):
        sum_vertice += median_x[j]*matriz_regioes[k][j]
      if (sum_vertice < 1):
        print("ERRO FALTANDO ESCOLA EM UMA REGIÃO")
        return False

    return True

In [ ]:
# Testa restrição
def verifica_restricoes_intermediario(median_x):
    # R6 {k in regioes_sjc}: sum{j in escolas}x[j]*escola_regiao_sjc[j,k] >= 1;
    for k in range(qtd_regioes):
      sum_vertice = 0
      for j in range(len(id_escolas)):
        sum_vertice += median_x[j]*matriz_regioes[k][j]
      if (sum_vertice < 1):
        print("ERRO FALTANDO ESCOLA EM UMA REGIÃO")
        return False

    return True

In [ ]:
def funcao_objetivo_intermediaria(median_x):
    sum_vertice = 0
    for j in range(len(median_x)):
      if (median_x[j] == 1):
        for i in range(len(id_vertices)):
          sum_vertice += matriz_distancias[j][i]
    return sum_vertice

In [ ]:
# minimize Z: sum{i in vertices, j in escolas} (distancias[i,j] * y[i,j]) + sum{j in escolas}custo_construir_sala*s[j];
def funcao_objetivo(median_x, median_y):
    sum_vertice = 0
    for j in range(len(median_x)):
      for i in range(len(id_vertices)):
        sum_vertice += matriz_distancias[j][i]*median_y[j][i]
    return sum_vertice

In [ ]:
# Melhorar: ao invés de fazer na ordem dos vértices, começar pelos mais próximos
def encontra_melhor_y(median):
    melhor_y = [[0 for i in range(len(id_vertices))] for j in range(len(id_escolas))]
    capacidades_alocacao = [float(x) for i,x in enumerate(matriz_capacidades)]
    for i in range(len(id_vertices)):
      min_dist = INF
      ind_dist = -1
      for j in range(len(id_escolas)):
        # Verifica se aquela escola é válida
        if (median[j] == 1 and (capacidades_alocacao[j] - 0.25*matriz_demanda[i]> 0)):
          # Verifica se aquela escolha melhora a distância
          if (matriz_distancias[j][i] < min_dist):
            min_dist = matriz_distancias[j][i]
            ind_dist = j
      if ind_dist == -1:
        print("ERRO NA HORA DE ENCONTRAR MELHOR Y")
        return False
      melhor_y[ind_dist][i] = 1
      capacidades_alocacao[ind_dist] -= 0.25*matriz_demanda[i]
    return melhor_y

In [ ]:
def generate_list_with_p_ones(length, p):
    # Create a list with p ones and the remaining as zeros
    ones = [1] * p
    zeros = [0] * (length - p)
    result = ones + zeros

    # Shuffle the list to make it random
    random.shuffle(result)

    return result

In [ ]:
# Solução inicial
k = 0
while k <= 1000:
  solucao_x = generate_list_with_p_ones(len(id_escolas), p)
  #solucao_y = encontra_melhor_y(solucao_x)
  #teste_restricoes = verifica_restricoes(solucao_x, solucao_y)
  teste_restricoes = verifica_restricoes_intermediario(solucao_x)
  print(k, teste_restricoes)
  if (teste_restricoes == False):
    k += 1
    continue
  else:
    break
if (teste_restricoes == False):
  print("---Erro: ultrapassou limite k sem encontrar solução---")

#funcao_objetivo_incumbente = funcao_objetivo(solucao_x, solucao_y)
funcao_objetivo_incumbente = funcao_objetivo_intermediaria(solucao_x)

In [ ]:
# Inicialização das variáveis
candidatos = [1 if i==0 else 0 for i in solucao_x]

In [ ]:
# ALGORITMO PRINCIPAL DE TROCAS
while candidatos.count(1) > 0:
  start = timer()
  indice_candidato = candidatos.index(1)
  candidatos[candidatos.index(1)] = 0
  houve_substituicao = False
  melhor_funcao_objetivo = funcao_objetivo_incumbente
  for i in range(len(id_escolas)):
    if (i != indice_candidato and solucao_x[i] == 1):
      solucao_teste_x = copy.deepcopy(solucao_x)
      solucao_teste_x[i] = 0
      solucao_teste_x[indice_candidato] = 1
      #solucao_teste_y = encontra_melhor_y(solucao_teste_x)
      #funcao_objetivo_teste = funcao_objetivo(solucao_teste_x, solucao_teste_y)
      funcao_objetivo_teste = funcao_objetivo_intermediaria(solucao_teste_x)
      if (funcao_objetivo_teste < melhor_funcao_objetivo):
        #solucao_teste_y = encontra_melhor_y(solucao_teste_x)
        #teste_restricoes = verifica_restricoes(solucao_teste_x, solucao_teste_y)
        teste_restricoes = verifica_restricoes_intermediario(solucao_teste_x)
        if (teste_restricoes == True):
          melhor_funcao_objetivo = funcao_objetivo_teste
          solucao_substituicao_x = copy.deepcopy(solucao_teste_x)
          #solucao_substituicao_y = copy.deepcopy(solucao_teste_y)
          houve_substituicao = True
  print(indice_candidato, houve_substituicao, melhor_funcao_objetivo, timer()-start)
  if (houve_substituicao):
    funcao_objetivo_incumbente = melhor_funcao_objetivo
    solucao_x = copy.deepcopy(solucao_substituicao_x)
    #solucao_y = copy.deepcopy(solucao_substituicao_y)

In [ ]:
solucao_y = encontra_melhor_y(solucao_x)

In [ ]:
verifica_restricoes(solucao_x,solucao_y)

In [ ]:
for i in range(len(id_vertices)):
  soma = 0
  for j in range(len(id_escolas)):
    soma += solucao_y[j][i]
  if soma == 0: print(i, soma)

In [ ]:
funcao_objetivo(solucao_x, solucao_y)

In [ ]:
solucao_escolas_vertices = pd.DataFrame.from_dict(solucao_y)

In [ ]:
id_map_dict = {i:x for i,x in enumerate(id_map)}

In [ ]:
solucao_escolas_vertices.columns = solucao_escolas_vertices.columns.map(id_map_dict)

In [ ]:
solucao_escolas_vertices.index = [x for i,x in enumerate(dicionario)]

In [ ]:
solucao_escolas_vertices

In [ ]:
solucao_escolas_vertices.to_csv("saida_escolas_vertices.csv", index_label='ID', sep=";")

In [ ]:
dicionario

In [ ]:
soma = 0
for i in range(len(id_vertices)):
  soma += solucao_y[dicionario['35121605']][i]
print(soma)

In [ ]:
soma = 0
for i in range(len(id_vertices)):
  soma += solucao_y[dicionario['35905124']][i]
print(soma)

In [ ]:
solucao_x[dicionario['35905124']]

In [ ]:
for k in range(qtd_regioes):
  if matriz_regioes[k][dicionario['35121605']] == 1: print(k)

In [ ]:
[x for i,x in enumerate(dicionario) if i == 62 or i == 94]

In [ ]:
matriz_regioes[7]